# 在word2vec上训练情感分析模型

In [5]:
import os
import re
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

from nltk.corpus import stopwords

from gensim.models.word2vec import Word2Vec

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans

### 和之前的操作一致

In [6]:
def load_dataset(name, nrows=None):
    datasets = {
        'unlabeled_train': 'unlabeledTrainData.tsv',
        'labeled_train': 'labeledTrainData.tsv',
        'test': 'testData.tsv'
    }
    if name not in datasets:
        raise ValueError(name)
    data_file = os.path.join('..', 'data', datasets[name])
    df = pd.read_csv(data_file, sep='\t', escapechar='\\', nrows=nrows)
    print('Number of reviews: {}'.format(len(df)))
    return df

In [7]:
eng_stopwords = set(stopwords.words('english'))

def clean_text(text, remove_stopwords=False):
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = text.lower().split()
    if remove_stopwords:
        words = [w for w in words if w not in eng_stopwords]
    return words

### 读入之前训练好的Word2Vec模型

In [8]:
model_name = '300features_40minwords_10context.model'
model = Word2Vec.load(os.path.join('..', 'models', model_name))

### 我们可以根据word2vec的结果去对影评文本进行编码

编码方式有一点粗暴，简单说来就是把这句话中的词的词向量做平均

In [9]:
df = load_dataset('labeled_train')
df.head()

Number of reviews: 25000


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"""The Classic War of the Worlds"" by Timothy Hin..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [10]:
def to_review_vector(review):
    words = clean_text(review, remove_stopwords=True)
    array = np.array([model[w] for w in words if w in model])
    return pd.Series(array.mean(axis=0)) #review编码只是简单的把每个词的word2vec进行求和

In [11]:
train_data_features = df.review.apply(to_review_vector)
train_data_features.head()

C:\MyPrograms\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\MyPrograms\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.008004,0.022376,-0.011617,0.005288,-0.004812,-0.010776,-0.009785,-0.019712,0.009523,0.019344,...,-0.004481,-0.010021,0.016328,0.012038,0.011507,0.000986,-0.007315,0.002268,-0.011729,-0.005051
1,-0.006026,0.031114,0.015922,-0.006982,-0.015125,-0.039399,-0.009210,-0.025657,-0.012569,0.004560,...,-0.010933,-0.016736,0.038783,-0.008356,0.024112,-0.002274,-0.000803,0.014396,0.012208,0.022051
2,-0.009216,0.031475,0.013193,-0.000996,-0.007406,-0.039634,-0.004208,-0.005805,-0.029545,0.001579,...,-0.015805,-0.017754,-0.005110,-0.005204,-0.002202,0.008102,0.016791,0.002761,-0.014985,0.025723
3,0.004692,0.037735,0.013596,0.006513,0.006973,-0.032882,0.000672,-0.023224,-0.016703,0.008977,...,-0.019670,-0.012493,0.000933,-0.001296,0.009946,0.007695,-0.000005,0.015884,0.006600,0.013251
4,-0.005244,0.027023,0.002643,0.000533,-0.005419,-0.032398,0.001182,-0.004578,-0.006754,0.012359,...,-0.015881,-0.012083,-0.008003,-0.008173,0.002846,0.014609,0.001996,0.002274,-0.013712,0.000170


### 用随机森林构建分类器

In [12]:
forest = RandomForestClassifier(n_estimators = 100, random_state=42)
forest = forest.fit(train_data_features, df.sentiment)

##### 同样在训练集上试试，确保模型能正常work

In [13]:
confusion_matrix(df.sentiment, forest.predict(train_data_features))

array([[12500,     0],
       [    0, 12500]])

### 清理占用内容的变量

In [14]:
del df
del train_data_features

### 预测测试集结果并上传kaggle

In [15]:
df = load_dataset('test')
df.head()

Number of reviews: 25000


,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [16]:
test_data_features = df.review.apply(to_review_vector)
test_data_features.head()

C:\MyPrograms\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\MyPrograms\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.002493,0.031020,0.013401,-0.011081,-0.006486,-0.043602,-0.026322,-0.036353,-0.029177,0.012026,...,-0.021319,-0.012728,-0.013037,0.006385,-0.014429,0.027920,0.017389,0.022579,-0.007292,0.006918
1,0.007193,0.020181,-0.014196,0.016120,-0.006612,0.004041,-0.031742,-0.028040,0.020427,0.033470,...,-0.007602,-0.009070,0.012854,0.000383,0.010604,0.002809,0.001015,0.001014,-0.009190,0.004522
2,0.012795,0.031792,-0.008013,0.013109,0.006368,-0.013133,-0.017673,-0.022258,0.008861,0.035568,...,-0.011096,-0.023866,0.016317,0.005704,0.006966,0.010968,-0.010528,0.014911,-0.016295,-0.018117
3,-0.006878,0.023271,0.009553,-0.000729,0.014761,-0.019271,-0.013603,-0.018781,-0.009137,0.015807,...,-0.024314,-0.018351,0.021872,-0.013081,0.007503,-0.001551,0.032887,0.016976,-0.006315,0.008143
4,-0.000929,0.028746,0.027761,0.011236,-0.007280,-0.042869,-0.005330,-0.015473,-0.017484,0.022764,...,-0.003449,-0.017091,0.023233,-0.002188,0.009607,-0.006031,0.000648,0.006987,-0.011551,0.002884


In [17]:
result = forest.predict(test_data_features)
output = pd.DataFrame({'id':df.id, 'sentiment':result})
output.to_csv(os.path.join('..', 'data', 'Word2Vec_model.csv'), index=False)
output.head()

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,1
3,7186_2,0
4,12128_7,1


In [18]:
del df
del test_data_features
del forest

------------------
### 对词向量进行聚类研究和编码
使用Kmeans进行聚类

In [39]:
# word_vectors = model.syn0
vec_words = model.wv.index2word
# print("word_vectors = ", word_vectors)
word_vectors = []
for w in vec_words:
    word_vectors.append([w, model[w]])
# num_clusters = word_vectors.shape[0] // 10
num_clusters = len(vec_words)//10
print("num_clusters = ", num_clusters)

num_clusters =  1305


C:\MyPrograms\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [41]:
%%time
kmeans_clustering = KMeans(n_clusters = num_clusters, n_jobs=4)
idx = kmeans_clustering.fit_predicbytearrayt(word_vectorsvectors)

Wall time: 0 ns


In [7]:
word_centroid_map = dict(zip(model.index2word, idx))

In [8]:
import pickle

filename = 'word_centroid_map_10avg.pickle'
with open(os.path.join('..', 'models', filename), 'bw') as f:
    pickle.dump(word_centroid_map, f)
    
#with open(os.path.join('..', 'models', filename), 'br') as f:
#    word_centroid_map = pickle.load(f)    

### 输出一些clusters看

In [9]:
for cluster in range(0,10):
    print("\nCluster %d" % cluster)
    print([w for w,c in word_centroid_map.items() if c == cluster])


Cluster 0
['praised', 'appreciated', 'noted', 'avoided', 'criticized', 'admired']

Cluster 1
['misfit', 'con', 'hoodlum', 'spy', 'rogue']

Cluster 2
['contrasts', 'healthy', 'glamour', 'eroticism', 'sensual']

Cluster 3
['matthew', 'kingsley', 'klein', 'hackman', 'meyers', 'perry', 'simpson', 'pullman', 'dana', 'olsen', 'ryan', 'barrie', 'caan', 'tho', 'farina', 'stiller', 'hutton', 'sparks', 'lillard', 'broderick', 'kline', 'reprise', 'mcconaughey', 'carvey', 'harrelson']

Cluster 4
['wolves', 'papillon', 'continent']

Cluster 5
['tick', 'drain', 'nailed', 'puke', 'boil', 'stalk']

Cluster 6
['cotton', 'denver', 'windsor', 'marsh', 'bell']

Cluster 7
['lighting', 'costumes', 'sfx', 'props', 'design', 'costuming', 'designs', 'makeup']

Cluster 8
['decline', 'swashbuckling', 'swashbuckler', 'prestige', 'potboiler', 'latter', 'glory', 'untouchables', 'fame']

Cluster 9
['slashed', 'butchered', 'mutilated', 'eaten', 'slaughtered', 'continually']


### 把评论数据转成cluster bag vectors

In [11]:
wordset = set(word_centroid_map.keys())

def make_cluster_bag(review):
    words = clean_text(review, remove_stopwords=True)
    return (pd.Series([word_centroid_map[w] for w in words if w in wordset])
              .value_counts()
              .reindex(range(num_clusters+1), fill_value=0))

In [12]:
df = load_dataset('labeled_train')
df.head()

Number of reviews: 25000


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"""The Classic War of the Worlds"" by Timothy Hin..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [13]:
train_data_features = df.review.apply(make_cluster_bag)
train_data_features.head()

,0,1,2,3,4,5,6,7,8,9,...,1296,1297,1298,1299,1300,1301,1302,1303,1304,1305
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,5,0,0,0,0,0,0,0,0


### 再用随机森林算法建模

In [14]:
forest = RandomForestClassifier(n_estimators = 100, random_state=42)
forest = forest.fit(train_data_features, df.sentiment)

##### 在训练集上试一试效果

In [15]:
confusion_matrix(df.sentiment, forest.predict(train_data_features))

array([[12500,     0],
       [    0, 12500]])

#### 去掉无用的占内存的量

In [16]:
del df
del train_data_features

### 载入测试数据做预测

In [17]:
df = load_dataset('test')
df.head()

Number of reviews: 25000


,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [18]:
test_data_features = df.review.apply(make_cluster_bag)
test_data_features.head()

,0,1,2,3,4,5,6,7,8,9,...,1296,1297,1298,1299,1300,1301,1302,1303,1304,1305
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
result = forest.predict(test_data_features)
output = pd.DataFrame({'id':df.id, 'sentiment':result})
output.to_csv(os.path.join('..', 'data', 'Word2Vec_BagOfClusters.csv'), index=False)
output.head()

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,1
3,7186_2,0
4,12128_7,1


In [ ]:
del df
del test_data_features
del forest